In [ ]:
import os
import cv2
import numpy as np
import pandas as pd
from tqdm import tqdm
from pathlib import Path
from natsort import natsorted
import glob
from gaze_tracking import GazeTracking

import tensorflow as tf

gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        tf.config.set_visible_devices(gpus[0], 'GPU')
        logical_gpus = tf.config.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPU")
    except RuntimeError as e:
        print(e)

In [ ]:
# Set your path
base_path = os.getcwd()
csv_path = os.path.join(base_path, 'folder_name', 'train-data-annotation-v1.csv')
features_path = os.path.join(base_path, 'folder_name', 'frame_folder_name')
output_path = os.path.join(base_path, 'folder_name', 'output_folder_name')

df = pd.read_csv(csv_path)
vid_names = df['VID_NAME'].unique()

# Initialize GazeTracking
gaze = GazeTracking()

def extract_gaze_features(image):
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    gaze.refresh(image_rgb)
    gaze_direction = gaze.horizontal_ratio()
    left_pupil = gaze.pupil_left_coords()
    right_pupil = gaze.pupil_right_coords()
    
    features = []
    features.append(gaze_direction if gaze_direction is not None else 0.5)
    features.extend(left_pupil if left_pupil is not None else [0, 0])
    features.extend(right_pupil if right_pupil is not None else [0, 0])
    
    return np.array(features)

for vid in tqdm(vid_names, desc="Processing VIDs"):
    vid_path = os.path.join(features_path, vid)
    vid_output_path = os.path.join(output_path, vid)
    
    if not os.path.exists(vid_path):
        print(f"Warning: Folder not found for {vid}")
        continue
    
    if not os.path.exists(vid_output_path):
        os.makedirs(vid_output_path)
    
    frame_paths = natsorted(glob.glob(os.path.join(vid_path, '*.jpg')))
    
    for i, fp in enumerate(frame_paths):
        im = cv2.imread(fp)
        if im is None:
            print(f"Failed to read image: {fp}")
            continue
        
        features = extract_gaze_features(im)
        
        base_name = os.path.splitext(os.path.basename(fp))[0]
        npz_path = os.path.join(vid_output_path, f"{base_name}.npz")
        
        np.savez_compressed(npz_path, features=features)
    
    print(f"Processed {len(frame_paths)} frames for {vid}")

print("Feature extraction completed for all VIDs.")